## Hand Gesture Recognition Models

### Imports and Utils

In [18]:
"""
Importing necessary libraries
"""

import cv2
import mediapipe as mp

import torch

# Initialize MediaPipe Imports
mp_drawings = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [57]:
"""
Defining Utiliy Functions
"""

def landmarks_to_list(image, multi_hand_landmarks) -> torch.Tensor:
    """
    image: The image on which the landmarks are detected
    multi_hand_landmarks: The landmarks of the hand

    Returns: torch.Tensor: The tensor of landmarks [x, y]
    """

    image_height, image_width, _ = image.shape

    landmarks_list = []
    if multi_hand_landmarks:
        for hand_landmarks in multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # orgin is top left corner (0, 0)
                landmarks_list.append([min(int(landmark.x * image_width), image_width - 1), -min(int(landmark.y * image_height), image_height - 1)])

    return torch.tensor(landmarks_list)

def normalize_landmarks(landmarks: torch.Tensor) -> torch.Tensor:
    """
    landmarks: torch.Tensor: The landmarks of the hand

    Returns: torch.Tensor: The normalized landmarks
    """

    landmarks = landmarks - landmarks[0]
    landmarks = landmarks / torch.max(landmarks)

    return landmarks

### Dataset Loading and Preprocessing

### Model Creation and Training

In [54]:
# Initialize MediaPipe Hands
HandLandmarker = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.5
)

# Initialize the webcam
cap = cv2.VideoCapture(0)

with HandLandmarker as landmarker:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # the BGR image to RGB.
        frame = cv2.flip(frame, 1)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Dectect the hand landmarks
        frame.flags.writeable = False
        results = landmarker.process(frame)

        # Draw the hand annotations on the image.
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawings.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawings.DrawingSpec(color=(97, 137, 48), thickness=2, circle_radius=4),
                    mp_drawings.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                )

        cv2.imshow('MediaPipe Hands', frame)
        if cv2.waitKey(5) & 0xFF == 27: # ESC
            break

cap.release()
cv2.destroyAllWindows()

In [58]:
# Convert the landmarks to a list wrt the image
landmarks_list = landmarks_to_list(frame, results.multi_hand_landmarks)
landmarks_list

tensor([[  72, -461],
        [ 150, -435],
        [ 213, -367],
        [ 251, -302],
        [ 284, -256],
        [ 178, -264],
        [ 204, -193],
        [ 215, -148],
        [ 224, -109],
        [ 134, -251],
        [ 157, -170],
        [ 170, -117],
        [ 181,  -72],
        [  89, -256],
        [ 101, -177],
        [ 114, -129],
        [ 128,  -87],
        [  41, -274],
        [  39, -214],
        [  44, -176],
        [  55, -138]])

In [59]:
# Normalize the landmarks
landmarks_normalized = normalize_landmarks(landmarks_list)
landmarks_normalized

tensor([[ 0.0000,  0.0000],
        [ 0.2005,  0.0668],
        [ 0.3625,  0.2416],
        [ 0.4602,  0.4087],
        [ 0.5450,  0.5270],
        [ 0.2725,  0.5064],
        [ 0.3393,  0.6889],
        [ 0.3676,  0.8046],
        [ 0.3907,  0.9049],
        [ 0.1594,  0.5398],
        [ 0.2185,  0.7481],
        [ 0.2519,  0.8843],
        [ 0.2802,  1.0000],
        [ 0.0437,  0.5270],
        [ 0.0746,  0.7301],
        [ 0.1080,  0.8535],
        [ 0.1440,  0.9614],
        [-0.0797,  0.4807],
        [-0.0848,  0.6350],
        [-0.0720,  0.7326],
        [-0.0437,  0.8303]])

### Testing and Plotting